In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
df_combined = pd.read_csv("combined_data.csv")

In [12]:
df_combined

,systemNumber,Column,Value
0,O187757,accessionNumber,C.326-1910
1,O185533,accessionNumber,683-1902
2,O151257,accessionNumber,3096-1852
3,O1757246,accessionNumber,WE.8354-2014
4,O160476,accessionNumber,420-1882
...,...,...,...
41373,O333534,Depth,27 cm
41374,O1733355,Width,105 mm
41375,O1718349,Width,93 mm
41376,O308541,Height,1.9 cm


In [13]:
selected_system_numbers = df_combined['systemNumber'].drop_duplicates().sample(n=5)
df_selected = df_combined[df_combined['systemNumber'].isin(selected_system_numbers)]

In [14]:
df_selected

,systemNumber,Column,Value
332,O187775,accessionNumber,C.333-1910
421,O187768,accessionNumber,C.332-1910
919,O71223,accessionNumber,2386&A-1901
1008,O195660,accessionNumber,414:1027-1885
1536,O336252,accessionNumber,459-1890
...,...,...,...
39702,O336252,materialsAndTechniques,NaN
40140,O187775,Height,19.1 cm
40168,O187768,Height,26.7 cm
40394,O195660,Height,8.6 cm


In [15]:
import pandas as pd

df = pd.read_csv('combined_data.csv') 

df_cleaned = df.dropna()

triples = []
for index, row in df_cleaned.iterrows():
    subject = row['systemNumber']
    predicate = row['Column']
    object_ = row['Value']
    triples.append((subject, predicate, object_))

print(len(triples))

32397


In [16]:
import random

random.seed(42)

system_numbers = list(set([item[0] for item in triples]))

random_system_numbers = random.sample(system_numbers, 5)

selected_triples = [item for item in triples if item[0] in random_system_numbers]

#print("随机抽取的 systemNumber:", random_system_numbers)
#  print("对应的元组:", selected_tuples)

In [17]:
selected_triples

[('O461572', 'accessionNumber', 'FE.40-1981'),
 ('O70237', 'accessionNumber', 'FE.73A-1982'),
 ('O115619', 'accessionNumber', 'FE.37-1978'),
 ('O230617', 'accessionNumber', '414:1264/A-1885'),
 ('O336240', 'accessionNumber', '1468-1892'),
 ('O461572', 'objectType', 'Bottle'),
 ('O70237', 'objectType', 'Dish'),
 ('O115619', 'objectType', 'Teapot'),
 ('O230617', 'objectType', 'Plaque'),
 ('O336240', 'objectType', 'Intaglio'),
 ('O461572', 'currentLocation_id', 'THES49865'),
 ('O70237', 'currentLocation_id', 'THES49865'),
 ('O115619', 'currentLocation_id', 'THES49877'),
 ('O230617', 'currentLocation_id', 'THES49875'),
 ('O336240', 'currentLocation_id', 'THES335012'),
 ('O461572', 'currentLocation_displayName', 'Ceramics, Room 145'),
 ('O70237', 'currentLocation_displayName', 'Ceramics, Room 145'),
 ('O115619',
  'currentLocation_displayName',
  'Ceramics, Room 136, The Curtain Foundation Gallery'),
 ('O230617',
  'currentLocation_displayName',
  'Ceramics, Room 138, The Harry and Carol Dj

In [18]:
from neo4j import GraphDatabase

In [19]:
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_node_and_relationship(self, system_number, attribute, value):
        try:
            with self.driver.session() as session:
                # 使用 attribute 作为关系类型
                session.run(f"""
                    MERGE (n:Entity {{systemNumber: $system_number}})
                    MERGE (v:Value {{value: $value}})
                    MERGE (n)-[:{attribute}]->(v)
                """, system_number=system_number, value=value)
        except Exception as e:
            print(f"Error creating node and relationship: {e}")

neo4j_conn = Neo4jConnection("bolt://localhost:7687", "neo4j", "ry123456")

In [20]:
for triple in selected_triples:
    neo4j_conn.create_node_and_relationship(triple[0], triple[1], triple[2])

neo4j_conn.close()

Error creating node and relationship: Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Error creating node and relationship: Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Error creating node and relationship: Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Error creating node and relationship: Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Error creating node and relationship: Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [E

MATCH (n)-[r]->(a) 

RETURN n, TYPE(r) AS relationship, properties(r), a

MATCH (n) 

DETACH DELETE n;

In [ ]:
class LLMClient:
    
    def __init__(self, host: str='36.137.79.97', port: int=30250):
        self.client = openai.Client(
            api_key="empty",
            base_url=f"http://{host}:{port}/v1")
        models = self.client.models.list()
        assert len(models.data) == 1
        self.model = models.data[0].id
        print("Use Model:", self.model)
    
    def chat(self, **kwargs):
        res = self.client.chat.completions.create(
            model=self.model, **kwargs)
        return [choice.message.content for choice in res.choices], dict(res.usage)

    def complete(self, **kwargs):
        res = self.client.completions.create(
            model=self.model, **kwargs)
        return [choice.text for choice in res.choices], dict(res.usage)

In [8]:
import pandas as pd
import openai

class LLMClient:
    def __init__(self, host: str='36.137.79.97', port: int=30250):
        self.client = openai.Client(
            api_key="empty",
            base_url=f"http://{host}:{port}/v1")
        models = self.client.models.list()
        assert len(models.data) == 1
        self.model = models.data[0].id
        print("Use Model:", self.model)
    
    def chat(self, **kwargs):
        res = self.client.chat.completions.create(
            model=self.model, **kwargs)
        return [choice.message.content for choice in res.choices], dict(res.usage)

    def complete(self, **kwargs):
        res = self.client.completions.create(
            model=self.model, **kwargs)
        return [choice.text for choice in res.choices], dict(res.usage)

# 生成缺失的 briefDescription
def generate_brief_description(sample: dict, model_client: LLMClient):
    prompt = (
        f"请基于以下文物信息生成一个简短的描述（约40个英文词左右）作为 briefDescription。\n"
        f"物品名称：{sample['primaryTitle']}\n"
        f"物品类型：{sample['objectType']}\n"
        f"制作日期：{sample['primaryDate']}\n"
        f"制作地点：{sample['primaryPlace']}\n"
        f"材料与工艺：{sample['materialsAndTechniques']}\n"
        f"物理描述：{sample['physicalDescription']}\n"
        f"请用英文简要描述这个文物，重点提及其外观、用途和文化背景。"
    )
    
    response, _ = model_client.complete(prompt=prompt, max_tokens=100)
    return response[0].strip()

# 处理所有样本并生成缺失的 briefDescription
def process_csv_file(input_file: str, output_file: str, model_client: LLMClient):
    # 读取原始 CSV 文件
    df = pd.read_csv(input_file, encoding='utf-8')

    # 打印列名，确认实际的列名
    print("CSV 文件的列名：", df.columns.tolist())

    # 遍历 DataFrame 中的每一行，生成并补充缺失的 briefDescription
    for index, row in df.iterrows():
        if pd.isna(row.get('briefDescription', None)):  # 如果 briefDescription 缺失
            print(f"缺少 briefDescription，正在为样本 {row['systemNumber']} 生成描述...")
            sample = row.to_dict()  # 将行转为字典，便于传递给生成函数
            try:
                df.at[index, 'briefDescription'] = generate_brief_description(sample, model_client)
            except KeyError as e:
                print(f"KeyError: 缺少列 '{e.args[0]}'，请检查列名是否正确")
                continue  # 如果出现 KeyError，则跳过该行
    
    # 将处理后的 DataFrame 保存为新的 CSV 文件
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"处理完成，新的 CSV 文件已保存为 {output_file}")

# 示例：调用方法，处理 CSV 文件
input_csv_file = 'wenwu.csv'  # 输入文件路径
output_csv_file = 'updated_wenwu.csv'  # 输出文件路径

# 创建 LLMClient 实例
model_client = LLMClient()

# 处理文件
process_csv_file(input_csv_file, output_csv_file, model_client)




Use Model: qwen2-72b-instruct
CSV 文件的列名： ['Unnamed: 0', 'systemNumber', 'accessionNumber', 'objectType', 'currentLocation_id', 'currentLocation_displayName', 'currentLocation_type', 'currentLocation_site', 'currentLocation_onDisplay', 'currentLocation_detail_free', 'currentLocation_detail_case', 'currentLocation_detail_shelf', 'currentLocation_detail_box', 'primaryTitle', 'primaryMaker_name', 'primaryMaker_association', 'primaryImageId', 'primaryDate', 'primaryPlace', 'images__primary_thumbnail', 'images__iiif_image_base_url', 'images__iiif_presentation_url', 'images_imageResolution', 'materialsAndTechniques', 'briefDescription', 'physicalDescription', 'marksAndInscriptions_0_content', 'marksAndInscriptions_0_inscriber_name_text', 'marksAndInscriptions_0_inscriber_name_id', 'marksAndInscriptions_0_inscriber_association_text', 'marksAndInscriptions_0_inscriber_association_id', 'marksAndInscriptions_0_date_text', 'marksAndInscriptions_0_date_earliest', 'marksAndInscriptions_0_date_latest

In [17]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# 读取数据
df = pd.read_csv('updated_wenwu.csv')  

# 清理数据：删除包含缺失值的行
df_cleaned = df.dropna(subset=['briefDescription'])

# 载入 Sentence 模型
sbert_model_path = '../pretrained/bge-large-en-v1.5'
model = SentenceTransformer(sbert_model_path)

def get_semantic_similarity(description: str, column_values: list):
    # 清理 column_values，确保它们都是字符串类型，并且忽略为空的值
    column_values = [str(val).strip() for val in column_values if str(val).strip() != '']
    
    if len(column_values) == 0:
        # 如果 column_values 为空，返回空列表，表示没有有效的数据进行相似度计算
        return []

    # 编码用户描述和列的每个值
    description_embedding = model.encode(description, convert_to_tensor=True)
    column_embeddings = model.encode(column_values, convert_to_tensor=True)

    # 计算语义相似度（余弦相似度）
    cosine_scores = util.pytorch_cos_sim(description_embedding, column_embeddings)[0]
    return cosine_scores

def get_top_n_similar_rows(description: str, column: str, df: pd.DataFrame, n=3, similarity_threshold=0.2):
    if column not in df.columns:
        print(f"Column '{column}' not found in DataFrame.")
        return []

    # 获取列值
    column_values = df[column].tolist()
    
    # 计算相似度
    similarity_scores = get_semantic_similarity(description, column_values)

    # 检查相似度数组是否为空
    if len(similarity_scores) == 0:
        print(f"Warning: No valid similarity scores for column '{column}' (empty or invalid values).")
        return []
    
    # 如果相似度最大值低于阈值，跳过该列
    if similarity_scores.max() < similarity_threshold:
        print(f"Warning: All similarity scores for column '{column}' are below the threshold.")
        return []

    # 将 similarity_scores 从 GPU 转移到 CPU
    similarity_scores_cpu = similarity_scores.cpu() if similarity_scores.is_cuda else similarity_scores

    # 获取前 N 个最相似的文物
    top_n_indices = similarity_scores_cpu.argsort(descending=True)[:n]
    top_n_rows = df.iloc[top_n_indices][['systemNumber', 'briefDescription', 'images__primary_thumbnail']]

    return top_n_rows

def display_results(top_n_rows):
    for idx, row in top_n_rows.iterrows():
        print(f"\nSystem Number: {row['systemNumber']}")
        print(f"Brief Description: {row['briefDescription']}")
        # 获取并展示图片链接（假设图片超链接存在于 'images__primary_thumbnail' 列）
        print(f"Image Link: {row['images__primary_thumbnail']}")
        print("=" * 40)

def get_relevant_artifacts(df: pd.DataFrame):
    description = input("请输入文物的描述信息：")
    
    # 获取与 'briefDescription' 列相似度最高的前三个文物
    top_n_rows = get_top_n_similar_rows(description, 'briefDescription', df, n=3)
    
    if not top_n_rows.empty:
        display_results(top_n_rows)
    else:
        print("未找到符合条件的文物。")

# 调用函数获取相关文物
get_relevant_artifacts(df_cleaned)



System Number: O187757
Brief Description: Yixing ware. Dish, in the form of a peach with peach flower relief decoration, stoneware with white glaze, China, Qing dynasty, 18th century
Image Link: https://framemark.vam.ac.uk/collections/2010EB8011/full/!100,100/0/default.jpg

System Number: O115610
Brief Description: Yixing ware. Square vase, red stoneware with bird and prunus blossom design in relief; China (Yixing), Qing dynasty, 1675-1725
Image Link: https://framemark.vam.ac.uk/collections/2010EB4170/full/!100,100/0/default.jpg

System Number: O115614
Brief Description: Yixing ware. Teapot with lid, stoneware with moulded relief of lotus and prunus design; China (Yixing), Qing dynasty, 18th century
Image Link: https://framemark.vam.ac.uk/collections/2010EB4229/full/!100,100/0/default.jpg


In [ ]:
# 使用 LLMClient 获取与前三个文物最相似的其他文物
def generate_similar_artifacts_description(top_3_artifacts, model_client: LLMClient):
    # 获取前三个文物的简短描述和 systemNumber
    descriptions = []
    system_numbers = []
    images = []  # 用于存储图片链接
    for idx, row in top_3_artifacts.iterrows():
        descriptions.append(row['briefDescription'])
        system_numbers.append(row['systemNumber'])
        images.append(row['images__primary_thumbnail'])
    
    # 构建输入给大模型的 prompt
    prompt = (
        "观察我给出的这三个文物，请根据它们的简短描述，您认为表格里与这三个文物最相似的前五个文物有哪些？\n"
        f"文物1 (System Number: {system_numbers[0]}): {descriptions[0]}\n"
        f"文物2 (System Number: {system_numbers[1]}): {descriptions[1]}\n"
        f"文物3 (System Number: {system_numbers[2]}): {descriptions[2]}\n"
        "请列出与这三个文物最相似的前五个文物，并简要说明它们的相似性。"
    )

    # 调用大模型进行推理
    response, _ = model_client.complete(prompt=prompt, max_tokens=200)
    
    return response[0].strip()

# 调用 LLMClient 生成推荐的最相似文物
model_client = LLMClient()

similar_artifacts_description = generate_similar_artifacts_description(top_3_similar_artifacts, model_client)

# 显示大模型的推荐结果，并包含 systemNumber 和图片链接
print("\n大模型认为与上述文物最相似的其他文物：")
print(similar_artifacts_description)

# 假设推荐的文物信息存储在变量 `recommended_artifacts`
recommended_artifacts = [
    {"systemNumber": "12350", "briefDescription": "A porcelain jar from the Ming dynasty with blue floral decorations.", "imageLink": "https://example.com/image1.jpg"},
    {"systemNumber": "12351", "briefDescription": "A Qing dynasty vase, similar in shape and pattern to the Tang vase.", "imageLink": "https://example.com/image2.jpg"},
    {"systemNumber": "12352", "briefDescription": "A small porcelain bowl from the Yuan dynasty with intricate blue patterns.", "imageLink": "https://example.com/image3.jpg"},
    {"systemNumber": "12353", "briefDescription": "A vase with similar designs from the Song dynasty.", "imageLink": "https://example.com/image4.jpg"},
    {"systemNumber": "12354", "briefDescription": "A Qing dynasty porcelain plate with blue and white decoration.", "imageLink": "https://example.com/image5.jpg"}
]

# 输出推荐文物的信息，包括 systemNumber 和图片链接
for artifact in recommended_artifacts:
    print(f"System Number: {artifact['systemNumber']}")
    print(f"Brief Description: {artifact['briefDescription']}")
    print(f"Image Link: {artifact['imageLink']}")
    print(f"Click to view image: {artifact['imageLink']}")
    print("=" * 40)


Use Model: qwen2-72b-instruct

大模型认为与上述文物最相似的其他文物：
我们有一份大的数据表，包含了很多文物的信息，但是目前只给出了以下五个潜在相似的文物：

| Catalog Number | Description                                                     |
|----------------|-----------------------------------------------------------------|
| WW1234         | A Kangxi period (1662-1722) stoneware dish, unglazed on the base, white staining on the inside modelled with plum blossom petals |
| WW2345         | A vase with a mottled blue-green and purple glaze, after the style of Ming dynasty Longquan ware |
| WW3456         | A 17th century Staffordshire lidded dish with beautiful scroll and shell decorations |
| WW4567         | A Qianlong period (1736-1796) Yixing ware vase with a mottled brown and blue glaze |
| WW5678         | A 18th century Wedgwood porcelain
System Number: 12350
Brief Description: A porcelain jar from the Ming dynasty with blue floral decorations.
Image Link: https://example.com/image1.jpg
Click to view image: https://example.com/image1.jpg
S